In [1]:
import numpy as np
import pandas as pd
from typing import List, Any, Dict, Tuple, Union

In [2]:
def entropy(ele: List[Any]) -> float:
    """计算列表的信息熵 (Empirical Entropy)

    [cite_start]对应书中代码清单 7-1 [cite: 77] [cite_start]及公式 (7-2) [cite: 45]。
    熵的计算公式：E(D) = -sum(p_k * log2(p_k))

    Args:
        ele (List[Any]): 包含类别取值的列表 (例如: ['是', '否', '是', ...])

    Returns:
        float: 信息熵值。
               如果列表为空或所有元素相同，熵应为 0。
    """
    # 1. 计算概率分布
    # set(ele) 获取唯一元素，count() 计算频次，除以总长度得到概率 P(x)
    probs = [ele.count(i)/len(ele) for i in set(ele)]

    # 2. 应用香农熵公式
    # H(x) = - Σ p(x) * log2(p(x))
    entropy = - np.sum([prob * np.log2(prob) for prob in probs])

    return entropy

In [3]:
def df_split(df: pd.DataFrame, col: str) -> Dict[Any, pd.DataFrame]:
    """根据指定特征的取值划分数据集

    [cite_start]对应书中代码清单 7-5 [cite: 202]。
    ID3 算法会对特征的每一个唯一取值建立一个分支。

    Args:
        df (pd.DataFrame): 待划分的训练数据 DataFrame。
        col (str): 划分数据的依据特征列名。

    Returns:
        Dict[Any, pd.DataFrame]: 划分后的数据字典。
            键 (Key): 特征的某个取值。
            值 (Value): 该特征取值为 Key 的数据子集 DataFrame。
    """
    unique_value = df[col].unique()
    res_dict = {}
    for key in unique_value:
        res_dict[key] = df[df[col] == key]
    return res_dict

In [4]:
def choose_best_feature(df: pd.DataFrame, label: str) -> Tuple[float, str, Dict[Any, pd.DataFrame]]:
    """根据信息增益选择最优特征

    计算公式：信息增益 g(D, A) = H(D) - H(D|A)

    Args:
        df (pd.DataFrame): 当前节点的训练数据。
        label (str): 标签列的名称 (Target variable)。

    Returns:
        Tuple[float, str, Dict]: 返回一个元组，包含：
            - max_value (float): 最大的信息增益值。
            - best_col (str): 信息增益最大的特征名称。
            - max_splited (Dict): 根据最优特征划分后的数据字典 (复用 df_split 的结果)。
    """

    # 1. 计算整个数据集 D 的经验熵 H(D)
    entropy_d = entropy(df[label].tolist())

    ig = {}
    splited_dict = {}
    # 2. 遍历每一个特征列
    for key, _ in df.items():
        # 跳过标签列
        if key == label:
            continue
        col_entropy = []
        p = []

        # a. 使用 df_split 划分数据
        splited_dict[key] = df_split(df, key)
        for value in splited_dict[key].values():
            col_entropy.append(entropy(value['play'].tolist()))
            p.append(len(value)/len(df))
        # b. 计算经验条件熵和信息增益
        ig[key] = entropy_d - np.sum(np.array([p]) * np.array([col_entropy]))

    # 4. 记录并返回增益最大的那个特征及其相关信息
    if len(ig) != 0:
        best_col = max(ig, key=ig.get)
        max_value = ig[best_col]
        max_splited = splited_dict[best_col]
    else:
        best_col = None
        max_value = None
        max_splited = None

    return max_value, best_col, max_splited

In [5]:
class ID3Tree:
    # 定义决策树结点类 [cite: 263]
    class TreeNode:
        def __init__(self, name):
            """
            初始化树结点

            Args:
                name: 结点名称（如果是内部结点，则是特征名；如果是叶子结点，则是分类结果）
            """
            # 1. 记录结点名字
            self.name = name
            # 2. 初始化连接字典 (connections)，用于存储子结点
            #    格式：{label: node}，其中 label 是特征取值 (如 '晴')，node 是对应的子结点对象
            self.connections = {}

        def connect(self, label, node):
            """
            建立当前结点与子结点的连接

            Args:
                label: 边上的标签（特征的取值，如 '晴'）
                node: 连接的子结点对象
            """
            # 将 label 和 node 存入 connections 字典
            self.connections[label] = node


    def __init__(self, df, label):
        """
        初始化 ID3 算法实例

        Args:
            df: 训练数据集
            label: 目标标签列名 (如 'play')
        """
        # 1. 保存 columns (特征列表，排除 label)
        self.columns = df.columns
        # 2. 保存 df 和 label
        self.df = df
        self.label = label
        # 3. 创建根结点 (self.root)，命名为 'Root'
        self.root = self.TreeNode("Root")

    def construct_tree(self):
        """
        开始构建决策树（对外调用的接口）
        """
        # 调用下面的递归函数 construct
        # 传入参数：当前父结点(self.root), 边标签(''), 数据集(self.df), 特征列表(self.columns)
        self.construct(self.root, '', self.df, self.columns)

    def construct(self, parent_node, parent_label, sub_df, columns):
        """
        递归构建决策树的核心逻辑 [cite: 284]

        Args:
            parent_node: 父结点对象
            parent_label: 指向当前结点的边标签（即父结点特征的某个取值）
            sub_df: 当前分支的数据子集
            columns: 当前可用的特征列表
        """
        # 1. 调用 choose_best_feature 选择最优特征
        max_value, best_col, max_splited = choose_best_feature(sub_df[columns], self.label)

        # 2. 处理特殊情况（递归停止条件）：
        # 如果选不到特征（best_feature 为空），或者数据纯度已经很高：
        if best_col is None:
            # -> 创建一个叶子结点（取 sub_df 中数量最多的类作为名字）
            node = self.TreeNode(sub_df[self.label].iloc[0])
            # -> 将其连接到 parent_node
            parent_node.connect(parent_label, node)
            return None

        # 3. 正常情况：
        #    -> 创建一个新的内部结点 (名字是 best_feature)
        node = self.TreeNode(best_col)
        #    -> 将其连接到 parent_node
        parent_node.connect(parent_label, node)

        # 4. 递归生成子树：
        #    -> 计算剩余特征 (new_columns = columns - best_feature)
        new_columns = [col for col in columns if col != best_col]
        #    -> 遍历 max_splited 中的每一个子集 (split_value, split_data)：
        #       调用 self.construct(当前结点, split_value, split_data, new_columns)
        for splited_value, splited_data in max_splited.items():
            self.construct(node, splited_value, splited_data, new_columns)

    def print_tree(self, node, tabs):
        print(tabs + node.name)
        for connection, child_node in node.connections.items():
            print(tabs + "\t" + "(" + connection + ")")
            self.print_tree(child_node, tabs + "\t\t")

In [10]:
# headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
df = pd.read_csv('https://raw.githubusercontent.com/w0330t/machine_learning_code_implementation/refs/heads/master/charpter7_decision_tree/example_data.csv', dtype={'windy': 'str'},)

In [11]:
tree1 = ID3Tree(df, 'play')
tree1.construct_tree()

In [12]:
tree1.print_tree(tree1.root, "")

Root
	()
		outlook
			(sunny)
				humility
					(high)
						temp
							(hot)
								windy
									(false)
										no
									(true)
										no
							(mild)
								windy
									(false)
										no
					(normal)
						temp
							(cool)
								windy
									(false)
										yes
							(mild)
								windy
									(true)
										yes
			(overcast)
				humility
					(high)
						temp
							(hot)
								windy
									(false)
										yes
							(mild)
								windy
									(true)
										yes
					(normal)
						temp
							(cool)
								windy
									(true)
										yes
							(hot)
								windy
									(false)
										yes
			(rainy)
				windy
					(false)
						humility
							(high)
								temp
									(mild)
										yes
							(normal)
								temp
									(cool)
										yes
									(mild)
										yes
					(true)
						humility
							(normal)
								temp
									(cool)
										no
							(high)
								temp
									(mild)
										no
